In [1]:
# prep output files to examine how much variability exists in the OceanRain data (OceanRain-R)
# NOW using machine learning approach that exists in dsd/dj
# bugs fixed 2/5/18 DD, now outputs numpy arrays of all lats, lons, and counts (within some bounds, rain-only)
# -- this is essentially pre-processing code for dsd/dj/OceanClustering, gmm_solo
#  updated DD 24/08/18 to keep total counts the same despite weighting by volume
# changed a few times aug/sept to use the OR "M" data instead, so as to pass along values per mm,
#  normalized in different ways that should be specified by output filename
# NOTE: inclusion of W files takes longer to read in and then subset... comment out 
#  if needs be, i.e. if there's no need for the ancillary vars

# cleaned up a little dec 2018... much of code isn't used anymore, but still useful for reading
#  through all OR ship-specific files and spitting out more easily used npy arrays for each variable of interest

dir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/'
#import h5py
from netCDF4 import Dataset # these data work with both h5py and netCDF4 libs?
import pygal
from pygal.style import DarkColorizedStyle,DarkSolarizedStyle,CleanStyle
from decimal import Decimal # display No in sci notation
import glob
from subprocess import call

tout = 'MSs3' # title out, used 'VS4' before for volume-weighted (R^3) and smoothed

In [2]:
from sklearn.decomposition import PCA
def pca2(data, pc_count = None):
    pca = PCA(n_components = npc) # can choose solver too, etc.
    return pca.fit_transform(data), pca.explained_variance_ratio_

npc = 6 # set number of PCs to solve for here
# input array should be 2 dimensional
#PCs, varex = pca2(np.transpose(darseas2d),npc)

def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y[1:], box, mode='same')
    y_smooth = np.append(y[0],y_smooth) # to leave first point unmolested
    return y_smooth

In [5]:
filist = glob.glob(dir+'*_M_*.nc') # raw (R) files -- SWITching TO M FILES
allct = []
alllo = []
allla = []
alldm,allrr,allnw = [], [], []
allku,allka, allmu,alldate = [], [], [], []

wfilist = glob.glob(dir+'*_W_*.nc') # adding W (water) files
alldt, allss, allepoch = [],[],[]

bn = np.load('oceanrain_binsizes.npy') # bin sizes in float array, copied to local dir from Dendrite DSD dir
# NOTE thesee are particle DIAMETERS, not radii
nplim = 50 # number particles limit to include data point
bnz = 80 # there are 128 (going up to 22mm!) so don't need them all, surely
startb = 12 # bin to start (most early ones have zero counts!)
smo = 5 # smoothing distance in OR bins
allcv = np.zeros([bnz,0])
allcr = np.zeros([bnz,0])
allcs = np.zeros([bnz,0])
allcw = np.zeros([bnz,0])

for f in wfilist[:]:
    tit = f[68:75]
    dat= Dataset(f)
    # vars used to screen data points for quality and rain:
    
for f in range(len(filist)): #filist[:]:
    ef1 = filist[f]
    ef2 = wfilist[f]
    tit = ef1[68:75]
    print(ef1[68:75],ef2[68:75])
    dat= Dataset(ef1)
    datw=Dataset(ef2) # W files
    # vars used to screen data points for quality and rain:
    pop= dat['probability_for_rain'][:]
    pf = dat['precip_flag'][:] # 0 = rain
    dm = dat['mass_weighted_mean_diameter_of_normalized_gamma'][:] #[rsub][psub]
    npar = dat['number_of_particles'][:]
    
    rsub = np.logical_and(pf==0, npar>nplim) #pop==1.0, npar>50)
    #rsub =  pf == 0   #, npar>nplim) #pop==1.0, npar>50)
    psub = pop[rsub] == 1.0  # ), dm[rsub] > 0.0)
    dsub = dm[rsub][psub] > 0.0
    
    lo = np.array(dat['longitude'])[rsub][psub][dsub]
    la = np.array(dat['latitude'])[rsub][psub][dsub]
    Nw = np.array(dat['intercept_of_normalized_gamma'])[rsub][psub][dsub]
    rr = np.array(dat['ODM470_precipitation_rate_R'])[rsub][psub][dsub]
    ku = np.array(dat['Ku_band_reflectivity'])[rsub][psub][dsub]
    ka = np.array(dat['Ka_band_reflectivity'])[rsub][psub][dsub]
    mu = np.array(dat['shape_parameter_of_normalized_gamma'])[rsub][psub][dsub]
    date= np.array(dat['date_UT'])[rsub][psub][dsub]
    dm = dm[rsub][psub][dsub]
    #npar = np.array(dat['number_of_particles'])[rsub][psub][dsub]
    nr = np.size(lo)
    print( nr, tit)
    #print('NPAR INFO: ',info(npar))
    
    # adding W variables:
    time_m, time_w = dat['unix_epoch'][:], datw['unix_epoch'][:] # need to match up since M is subset of W!
    inds = [a in time_m for a in time_w] # indices for M in W data (unix time should be unique!)
    dt = np.array(datw['dew_point_temperature'][inds])[rsub][psub][dsub]
    ss = np.array(datw['sea_surface_temperature'][inds])[rsub][psub][dsub]
    
    rallcts = np.zeros([bnz,nr])  # all counts
    sallcts = np.zeros([bnz,nr])  # smoothed all counts
    vallcts = np.zeros([bnz,nr])  # weighted by volume/mass (R^3) [then smoothed]
    wallcts = np.zeros([bnz,nr])  # weighted by volume/mass (R^3)

    for b in range(bnz):
        # values for bins given are size CENTERs, as said in README of documentation
        bz = np.array(dat['bin'+str(b+startb+1)])[rsub][psub][dsub] # integer counts, array size is ntime
        rallcts[b,:] = bz[:]
        wallcts[b,:] = bz[:]*((bn[b+startb]*0.5)**3) # weighting raw counts by r^3 (volume/rwc)
        #wallcts[b,:] =bz[:]*(bn[b+startb]**3)*(bn[b+startb]*2)**0.67 # counts weighted by volume/mass/rwc * velocity
    
    for n in range(nr):
        # smooth cts with moving boxcar, but don't change total number of counts
        ## normalizing below anyway, so non-int count values are fine
        vallcts[:,n] = smooth(wallcts[:,n],smo) * (np.sum(wallcts[:,n])/np.sum(smooth(wallcts[:,n],smo)))
        # should conserve total mass (instead of counts) for each raining minute
        sallcts[:,n] = smooth(rallcts[:,n],smo) * (np.sum(rallcts[:,n])/np.sum(smooth(rallcts[:,n],smo)))
    
    #tot_drops = np.sum(rallcts, axis=0)
    #gu = np.array(np.where(tot_drops > nplim))[0,:] 
    #gu = np.array(np.where(npar > nplim))[0,:] 
    # screen out points with few drops -- CURRENTLY DOESNT DO ANYTHING FOR M RUNS>>>
    #tot_drops_g = tot_drops[gu]
    rallcts_g = rallcts #[:,gu] # save raw counts for eliminating obs with few drops
    vallcts_g = vallcts #[:,gu]
    wallcts_g = wallcts #[:,gu]
    sallcts_g = sallcts #[:,gu]
    #print(np.shape(tot_drops),np.shape(sallcts_g))
    
    alllo.extend(lo)#[gu])    
    allla.extend(la)#[gu])    
    alldm.extend(dm)#[gu])
    allrr.extend(rr)#[gu])
    allnw.extend(Nw)#[gu])
    allku.extend(ku)#[gu])
    allka.extend(ka)#[gu])
    allmu.extend(mu)#[gu])
    
    allss.extend(ss)
    alldt.extend(dt)
    alldate.extend(date)
    allepoch.extend(time_m) # saving unix epoch

    #nrn = np.size(tot_drops_g)
    allcv = np.hstack([allcv,vallcts_g])
    allcr = np.hstack([allcr,rallcts_g])
    allcs = np.hstack([allcs,sallcts_g])
    allcw = np.hstack([allcw,wallcts_g])
        #allcw = np.append(allcw, np.resize(wallcts_g[:,n],[bnz,1]), axis=1)
    
np.save('dj/data/alllatd'+tout, np.array(allla)) # one numpy dump for all (ship) files
np.save('dj/data/alllond'+tout, np.array(alllo))
np.save('dj/data/allrrd'+tout, np.array(allrr))
np.save('dj/data/allnwd'+tout, np.array(allnw))
np.save('dj/data/alldmd'+tout, np.array(alldm))
np.save('dj/data/allkud'+tout, np.array(allku))
np.save('dj/data/allkad'+tout, np.array(allka))
np.save('dj/data/allmud'+tout, np.array(allmu))
np.save('dj/data/alldatd'+tout,np.array(alldate))
np.save('dj/data/allssd'+tout, np.array(allss))
np.save('dj/data/alldtd'+tout, np.array(alldt))
np.save('dj/data/allepochd'+tout, np.array(allepoch))

# normalize in the vertical by counts, integrate to 1.0
vnallcout = allcv / np.sum(allcv, axis=0) # normalized, vol-weighted, smoothed
wallcout  = allcw / np.sum(allcw, axis=0) # normalized, vol-weighted
sallcout  = allcs / np.sum(allcs, axis=0) # normalized, smoothed (not weighted!)
print(np.sum(vnallcout[:,174])) # should be 1!

# trying to normalize by RWC:
mpy = bn[startb:(startb+bnz+1)]
dD = [mpy[n+1]-mpy[n] for n in range(len(mpy[:bnz]))] # bin width [diameter, mm]
norm_fac = 1000.0 * (4/3 * np.pi * (0.001*mpy[:-1]*.5)**3) * dD
# density * volume * width of bin (since per mm from M data) [units: kg/m3 * m3... ]
rwc_norm = np.array([norm_fac * allcs[:,s] / np.sum(norm_fac * allcs[:,s]) for s in range(len(alllo))]).transpose()

np.save('dj/data/normallcts'+tout+'.'+str(bnz),vnallcout) # normalized to 1.0 sum 
#np.save('dj/data/normallcts'+tout+'.'+str(bnz),wallcout) # weighted, normalized but not smoothed 
np.save('dj/data/snallcts'+tout+'.'+str(bnz),sallcout) # normalized, smoothed, not weighted 
np.save('dj/data/vallcts'+tout+'.'+str(bnz),allcv) # not normalized (but still volume-weighted, smoothed)
np.save('dj/data/rallcts'+tout+'.'+str(bnz),allcr) # not normalized, just raw counts
np.save('dj/data/sallcts'+tout+'.'+str(bnz),allcs) # not normalized, just smoothed raw counts
np.save('dj/data/wallcts'+tout+'.'+str(bnz),allcw) # not normalized, just weighted raw counts
np.save('dj/data/rwallcts'+tout+'.'+str(bnz),rwc_norm) # RWC-normalized, (smoothed) raw counts
print(np.shape(vnallcout),np.shape(allla))


The-Wor The-Wor
1180 The-Wor
Investi Investi
10242 Investi
Maria-S Maria-S
9392 Maria-S
Meteor_ Meteor_
10461 Meteor_
Polarst Polarst
32870 Polarst
Roger-R Roger-R
4750 Roger-R
SonneI_ SonneI_
3277 SonneI_
SonneII SonneII
19944 SonneII


NameError: name 'andstophere' is not defined

In [ ]:
print(info(alldat))

print(info(allku))
print(info(allka))
print(info(allmu))
print(rwc_norm[:40,5])
print(rwc_norm[:40,259])
print(rwc_norm[:40,9990+300])
print(rwc_norm[:40,9990-300])
print(np.sum(rwc_norm[:,2159]))
print(np.sum(rwc_norm[:,259]))

In [ ]:
cz = 9990
rg = 40
#print('raw ',allcr[:rg,cz])
#print('raww ',(bn[startb:(startb+rg)]**3)*allcr[:rg,cz]) # should be identical!
#print('wghtd ',allcw[:rg,cz])
#print('smoothd ',allcs[:rg,cz])
#print('wghtd smthd ',allcv[:rg,cz])
#print('vol smoothd normd ',vnallcout[:20,0])
print('totalz: ',np.sum(allcr[:rg,cz]),np.sum(allcs[:rg,cz]),np.sum(allcw[:rg,cz]),np.sum(allcv[:rg,cz]))
plt.figure(figsize=[8,8])
#plt.plot(range(rg),    allcr[:rg,cz],'r',label='raw')
plt.plot(range(rg),    allcw[:rg,cz],'r--',label='wgtd')
#plt.plot(range(rg),    allcs[:rg,cz],'b',label='sm raw')
plt.plot(range(rg),    allcv[:rg,cz],'k',label='vol smthd')
plt.plot(range(rg),vnallcout[:rg,cz],'g',label='vol smthd norm')
plt.plot(range(rg), wallcout[:rg,cz],'c',label='vol norm')
plt.plot(range(rg), rwc_norm[:rg,cz],'o',label='RWC norm')
plt.plot(range(rg), rwc_norm[:rg,cz+300],'o',label='RWC norm')
plt.plot(range(rg), rwc_norm[:rg,cz-300],'o',label='RWC norm')
plt.plot(range(rg), rwc_norm[:rg,cz-3800],'o',label='RWC norm')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
# how many are in each lat bin, at a few resolutions:
bznss = 15.0 # degrees lat
maxlat, minlat = 80, -60
ny = int((maxlat - minlat)/bznss)
latz,dmz = np.array(allla), np.array(alldm)
lonz,rrz,nwz = np.array(alllo), np.array(allrr), np.array(allnw)
kuz,kaz,muz = np.array(allku), np.array(allka), np.array(allmu)
zave = np.zeros(ny)
for y in range(ny):
    lasub = np.logical_and( latz >= minlat+bznss*y, latz < minlat+(bznss*(y+1)))
    print(np.size(latz[lasub]),minlat+bznss*y,minlat+bznss*(y+1))
    zave[y] = np.size(latz[lasub])
#print(np.mod(zave,float(np.min(zave))))
print(np.round(zave/float(np.min(zave))))
multiple = np.round(zave/float(np.min(zave))).astype(int) 
#determine fraction that'll yield approx the same number in each lat range
print(multiple) #i.e. take every _th point to get about equal #s per lat bin

# shrink size of output arrays relative to smallest lat band for counts:
shrnklat,shrnklon = [],[]
shrnkrr,shrnknw,shrnkdm = [],[],[]
shrnkku,shrnkka,shrnkmu = [],[],[]
shrnkctsn = np.zeros([bnz,0])
shrnkcts  = np.zeros([bnz,0])
for y in range(ny):
    lasub = np.logical_and( latz >= minlat+bznss*y, latz < minlat+(bznss*(y+1)))
    shrnklat.extend(latz[lasub][::multiple[y]])
    shrnklon.extend(lonz[lasub][::multiple[y]])
    shrnkrr.extend(rrz[lasub][::multiple[y]])
    shrnknw.extend(nwz[lasub][::multiple[y]])
    shrnkdm.extend(dmz[lasub][::multiple[y]])
    shrnkku.extend(kuz[lasub][::multiple[y]])
    shrnkka.extend(kaz[lasub][::multiple[y]])
    shrnkmu.extend(muz[lasub][::multiple[y]])
    addctsn = vnallcout[:,lasub] # normalized
    addcts = allcv[:,lasub] # non-normalized (still weighted)
    shrnkctsn = np.append(shrnkctsn, addctsn[:,::multiple[y]], axis=1)
    shrnkcts  = np.append(shrnkcts,  addcts[:,::multiple[y]], axis=1)
print(np.size(shrnklat), ny*np.min(zave), 'min:',np.min(zave) )

# save these shrunken arrays with different filenames to differentiate
np.save('dj/data/shrnk.alllatd'+tout, np.array(shrnklat)) # one numpy dump for all files
np.save('dj/data/shrnk.alllond'+tout, np.array(shrnklon))
np.save( 'dj/data/shrnk.allrrd'+tout, np.array(shrnkrr))
np.save( 'dj/data/shrnk.alldmd'+tout, np.array(shrnkdm))
np.save( 'dj/data/shrnk.allnwd'+tout, np.array(shrnknw))
np.save( 'dj/data/shrnk.allkud'+tout, np.array(shrnkku))
np.save( 'dj/data/shrnk.allkad'+tout, np.array(shrnkka))
np.save( 'dj/data/shrnk.allmud'+tout, np.array(shrnkmu))

# shrunk cts (~equal in each lat bin), V is for volume weighted counts
np.save('dj/data/shrnk.normallcts'+tout+'.'+str(bnz),shrnkctsn) # normalized to 1.0 sum (above)
np.save('dj/data/shrnk.allcts'+tout+'.'+str(bnz),shrnkcts) # raw counts (but still weighted by R^3)

In [ ]:
print(bn[bnz+startb]) # biggest size bin passed along

In [ ]:
print(lkjasdflkjsdf) # not using this section now

# take bin closest to Dm, set to counts in bin __ (10?), reorganize counts into X other bins
#import rebin
from scipy import interpolate, ndimage

newbnz = 30 # number new bins... set dm in middle kinda -- set in concert with newbnz frac below
bnm = bn[startb:(startb+bnz)]
newb = np.arange(bnm[0],bnm[bnz-1],(bnm[10]-bnm[9])*.1) # static, interpolating original cts
#print(bnm.size,bnm)
#print(alldm[0:30])
dm_in = shrnkctsn #allc # was allc before, trying shrunken lat-based array now
dout = np.zeros([newbnz,np.shape(dm_in)[1]]) #len(allla)])
for c in range(np.shape(dm_in)[1]): #[0:9]:
    #print('close?',bnm[bbin[0]],alldm[c]) # make sure closest bin name and dm match closely
    # next interpolate to better populate counts array for below rebinning
    f = interpolate.interp1d(bnm, dm_in[:,c] ) #allc[:,c])
    allci = f(newb) #np.arange(bnm[0],bnm[bnz-1],(bnm[20]-bnm[19])*.1))
    allci = ndimage.filters.gaussian_filter1d(allci,sigma=4)
    #plt.figure(figsize=[12,8])
    #plt.plot(bnm,allc[:,c],label='c')
    #plt.plot(newb,allci,label='ci')
    #plt.xlim(0.3,2.0)
    #plt.legend()
    
    dif = np.array(alldm[c] - newb) #bnm) # first take dif to find closest bin to dm
    bbin = np.where(abs(dif) == np.min(abs(dif)))[0]
    xnewl = np.arange(newb[0], newb[bbin[0]], (newb[bbin[0]]-newb[0])/(newbnz*.6) ) ##coord with newbnz
    xnewh = np.arange(newb[bbin[0]],newb[-1], (newb[-1]-newb[bbin[0]])/(newbnz*.4))
    #xnewl = np.arange(bnm[0], bnm[bbin[0]], (bnm[bbin[0]]-bnm[0])/(newbnz*.75) )
    #xnewh = np.arange(bnm[bbin[0]],bnm[bnz-1], (bnm[bnz-1]-bnm[bbin[0]])/(newbnz*.25))
    # cram above Dm counts into newbnz/2 bins
    #print('new',newb)
    #print('low',xnewl)
    #print('hi',xnewh)
    fnew = interpolate.interp1d(newb,allci)
    newc = np.append( fnew(xnewl), fnew(xnewh) )
    #print(len(fnew(xnewl)),len(fnew(xnewh)))
    #print('hic',fnew(xnewh))
    #print('cts old',np.sum(allci),allci[0:120])
    #print('cts new',np.size(newc),np.sum(newc),newc)
    if np.sum(newc) == 0: 
        print('zero!?',alldm[c],bbin,np.sum(allci),np.sum(allc[:,c]))
        print(xnewl,xnewh)
        print('old',allc[:,c])
        print('int',np.mean(allci),np.max(allci[:]))
        print(xnewl,fnew(xnewl))
        #print('new',newc)
        newc = dout[:,c-2] # some bullshit workaround
    else:
        newc *= np.sum(allc[:,c])/np.sum(newc)
    #print('cts adj',newc)
    #print('sumz',np.sum(newc),np.sum(allc[:,c]))
    dout[:,c] = newc[0:newbnz]
    
plt.show()


#allcout = allc / np.sum(allc,axis=0) # normalize in the vertical by counts, integrate to 1.0
dallcout = dout / np.sum(dout,axis=0) # normalize in the vertical by counts, integrate to 1.0
    
np.save('dj/data/dmnormctsV.'+str(newbnz),dallcout) # normalized by dm
#np.save('dj/data/normallcts',allcout)
#print('do deez match')
#print(np.shape(alllo),np.shape(dout))


In [ ]:
#pcs, varex = pca2(lallcts,npc)
#print(bnm[50:60])
#print(np.shape(allcts))
#print(info(dout))
#print(dout[:,0])
#print(allc[:,0])
#print(dout[:,990])
#print(allc[:,990])
#r1,r2,r3 = 743, 12345, 19123
r1,r2,r3 = 43, 2345, 1912
print(alldm[r1],alldm[r2])
print(allc[:,r2])
plt.figure(figsize=[8,5])
exx = np.arange(0,len(allc[:,0]))
plt.plot(exx,allc[:,r1],label='r1')
plt.plot(exx,allc[:,r2],label='r2')
#plt.plot(exx,allc[:,r3],label='r3')
plt.legend()
plt.figure(figsize=[8,5])
plt.plot(np.arange(0,newbnz),dout[:,r1],label='r1d')
plt.plot(np.arange(0,newbnz),dout[:,r2],label='r2d')
#plt.plot(np.arange(0,newbnz),dout[:,r3],label='r3d')
plt.legend()
plt.show()

In [ ]:
#print(np.shape(pcs))
#print(pcs[:,0])
#print(var
samma = np.sum(allc,axis=0)
print(np.shape(samma))
print(samma[0:20])

In [ ]:
##f1 = plt.figure(figsize=[8,8])
##plt.plot(ex,pcs[:,0],)
##print(np.shape(bname),bname)
#pcp = pygal.Line(title='PCs',x_title='Size bins',y_title='Counts',x_labels=bname,\
#                    style=DarkSolarizedStyle,legend_at_top=True)
##pcp.logarithmic = True
##pcp.x_label_major_every = 5
##pcp.x_label_rotation = 30
##pcp.show_minor_x_labels = False
#varst = []
#for p in range(npc):
#    varst.extend(["{:.2}".format(Decimal(varex[p]*100.0))])
#print(varst)
#pcp.add('PC1: '+varst[0], pcs[:,0])
#pcp.add('PC2: '+varst[1], pcs[:,1])
#pcp.add('PC3: '+varst[2], pcs[:,2])
#pcp.add('PC4: '+varst[3], pcs[:,3])
##pcp.add('PC5', pcs[:,4])
#pcp.render_to_png('test-inv.png')
#
#
#
#

In [ ]:
#print(pcs[:,2])

In [ ]:
print(info(shrnkctsn))
print(np.shape(shrnkctsn))
